In [1]:
from google.cloud import bigquery
from dateutil import tz
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from google.cloud.bigquery import QueryJob, Client

The method of construction will vary based on arguments passed during construction: 

- if given a QueryJob object, then construct Job by inheriting all properties of QueryJob object.
- if given positional arguments or keyword arguments, then construct Job via assigning positional arguments in the following order (name, last_state, last_checked) or keyword arguments to those respective argument names. Inherit all properties of QueryJob object, but force the client property to be None.
- if given both a QueryJob object and positional arguments and/or keyword arguments, then construct Job by first inheriting all properties of the QueryJob then assign positional arguments and keyword arguments to their respective properties.

Do not require arguments or keyword arguments in creating the Job.

It should pass the following tests:

# Create with instance of QueryJob from library
job = Job(jobs[0])

# Create with name and nothing else
job = Job('asdf')

# Create with instance of QueryJob from library
job = Job(query_job)

# Create with instance of QueryJob from library and with name
job = Job(query_job, 'asdf')

In [11]:
class Job(QueryJob):
    def __init__(self, *args, **kwargs):
        if args and isinstance(args[0], QueryJob):
            super().__init__(self, QueryJob)
            if args[1:] or kwargs:
                self._assign_properties(*args[1:], **kwargs)
        else:
            self.name = kwargs.get('name')
            self.last_state = kwargs.get('last_state')
            self.last_checked = kwargs.get('last_checked')
    
    def _assign_properties(self, *args, **kwargs):
        property_names = ['name', 'last_state', 'last_checked']
        
        for i, value in enumerate(args):
            setattr(self, property_names[i], value)
        
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def __repr__(self):
        return ', '.join(self.name, self.job_id)
    
    def __str__(self):
        user_string = f"User: {self.user_email}"
        created_string = f"Created: {self.created.astimezone(tz.tzlocal()).strftime('%b %d, %Y %I:%M %p')}" if self.created else None
        last_checked_string = f"Last Checked: {self.last_checked.astimezone(tz.tzlocal()).strftime('%b %d, %Y %I:%M %p')}" if self.last_state else None
        if self.error:
            return "\t | \t".join([self.name, self.job_id, user_string, created_string, last_checked_string, self.error['message']])
        else:
            return "\t | \t".join([self.name, self.job_id, user_string, created_string, last_checked_string, self.last_state])

class UserQueryManager:
    def __init__(self):
        pass
        
    def get_active(self):
        to_zone = tz.tzlocal()
        client = bigquery.Client()
        self.active_jobs = [job for job in client.list_jobs(state_filter="RUNNING")]
        return self.active_jobs

In [7]:
manager = UserQueryManager()
jobs = manager.get_active()

In [8]:
jobs[0]

QueryJob<project=nbcu-ds-sandbox-a-001, location=US, id=5c782236-ff2d-43ca-b38c-93cdd4e088bc>

# Tests

In [9]:
# Create empty job
job = Job()
job

TypeError: __init__() missing 1 required positional argument: 'query'

In [ ]:
# Create with name and nothing else
job = Job('asdf')
job

In [ ]:
# Create with instance of QueryJob from library
job = Job(jobs[0])
job

In [ ]:
# Create with instance of QueryJob from library and with name
job = Job(jobs[0], 'asdf')